In [1]:
import psycopg2
import os
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
# Connect to Redshift
conn = psycopg2.connect(
    dbname=os.getenv('REDSHIFT_SCHEMA'),
    user=os.getenv('REDSHIFT_USER'),
    password=os.getenv('REDSHIFT_PASSWORD'),
    host=os.getenv('REDSHIFT_HOST'),
    port=5439
)

cur = conn.cursor()

# Example schema — adjust columns to match your CSV
create_table_sql = """
DROP TABLE IF EXISTS DAILY_CRIPTO_PRICES;

CREATE TABLE DAILY_CRIPTO_PRICES (
    usd FLOAT,
    cripto VARCHAR(100),
    date TIMESTAMP
);
"""

cur.execute(create_table_sql)
conn.commit()

#cur.close()

In [3]:
import pandas as pd

df = pd.read_sql("SELECT * FROM DAILY_CRIPTO_PRICES;", conn)
print(df.head())

/var/folders/1k/sh8hpzz512gfjrxqfmzwshbc0000gn/T/ipykernel_5475/2122793251.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql("SELECT * FROM DAILY_CRIPTO_PRICES;", conn)


Empty DataFrame
Columns: [usd, cripto, date]
Index: []


In [4]:
import pandas as pd 
prices_historic = pd.read_csv('~/Downloads/btc-usd-max.csv',
    dtype={"total_volume": "float64", "price": "float64","market_cap":"float64"},
    parse_dates=["snapped_at"],
    date_parser=lambda x: pd.to_datetime(x, utc=True).strftime("%d-%m-%Y"))

/var/folders/1k/sh8hpzz512gfjrxqfmzwshbc0000gn/T/ipykernel_5475/2609599766.py:2: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  prices_historic = pd.read_csv('~/Downloads/btc-usd-max.csv',
/var/folders/1k/sh8hpzz512gfjrxqfmzwshbc0000gn/T/ipykernel_5475/2609599766.py:2: UserWarning: Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  prices_historic = pd.read_csv('~/Downloads/btc-usd-max.csv',


In [5]:
prices_historic.columns = ['date', 'usd', 'market_cap', 'total_volume']
prices_historic = prices_historic[['date', 'usd']]
prices_historic['cripto'] = 'bitcoin'

In [39]:
pd.to_datetime("01-12-2025",format="%d-%m-%Y")

Timestamp('2025-12-01 00:00:00')

In [5]:
from sqlalchemy import create_engine, text

engine = create_engine(os.getenv("REDSHIFT"))

# Insert a single row
with engine.connect() as conn:
    conn.execute(
        text("INSERT INTO DAILY_CRIPTO_PRICES (usd, cripto,date) VALUES (:usd, :cripto,:date)"),
        {"usd": 10010.094, "cripto":'bitcoin','date':pd.to_datetime("02-12-2025",format="%d-%m-%Y")}
    )

/var/folders/1k/sh8hpzz512gfjrxqfmzwshbc0000gn/T/ipykernel_5475/4007788165.py:7: RemovedIn20Warning: Deprecated API features detected! These feature(s) are not compatible with SQLAlchemy 2.0. To prevent incompatible upgrades prior to updating applications, ensure requirements files are pinned to "sqlalchemy<2.0". Set environment variable SQLALCHEMY_WARN_20=1 to show all deprecation warnings.  Set environment variable SQLALCHEMY_SILENCE_UBER_WARNING=1 to silence this message. (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  conn.execute(
